In [9]:
from ann import cost, cost_regL2, backprop, backprop2, predict, feedForward
from utils import checkNNGradients

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.io import loadmat, savemat
from scipy.optimize import minimize
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

testData = pd.read_csv("winequality-test.txt", sep=',', header=None)
trainData = pd.read_csv("winequality-train.txt", sep=',', header=None)

X_train = trainData.iloc[:, :-1]
y_train = trainData.iloc[:, -1:]

y_test = testData.iloc[:, -1:]
X_test = testData.iloc[:, :-1]

scaler = StandardScaler()

# Ajustar y transformar los datos
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=8)

encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train)

layer_sizes = [X_train.shape[1], 100, y_train.shape[1]]
epsilon = 0.12
theta_list = []

for i in range(len(layer_sizes) - 1):
    theta_list.append(np.random.rand(layer_sizes[i + 1], layer_sizes[i] + 1) * 2 * epsilon - epsilon)

iterations = 1000
lambda_ = 1
alpha = 0.1

for i in range(iterations):
    cost, grads = backprop(theta_list, X_train, y_train, lambda_)

    for j in range(len(theta_list)):
        theta_list[j] -= alpha * grads[j]

predictions = predict(theta_list, X_train)

# Datos entrenamiento
accuracy = accuracy_score(np.argmax(y_train, axis=1), predictions)
print(f'Precisión del perceptrón: {accuracy * 100:.2f}%')

conf_matrix = confusion_matrix(np.argmax(y_train, axis=1), predictions)
print(f'Matriz de confusión:\n {conf_matrix}')

mse = mean_squared_error(np.argmax(y_train, axis=1), predictions)
print(f'Error cuadratico medio: {mse:.2f} \n')

# Datos reales
predictions = predict(theta_list, X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Precisión del perceptrón: {accuracy * 100:.2f}%')

conf_matrix = confusion_matrix(y_test, predictions)
print(f'Matriz de confusión:\n {conf_matrix}')

mse = mean_squared_error(y_test, predictions)
print(f'Error cuadratico medio: {mse:.2f} \n')


# report_sklearn = classification_report(y_test, predictions)
# print(report_sklearn)

# print("\nExplicaciones:")
# print("Precision: Es la proporción de instancias clasificadas como positivas que son realmente positivas.")
# print("Recall: Es la proporción de instancias positivas que fueron correctamente clasificadas como positivas.")
# print("F1-score: Es la media armónica ponderada de precision y recall. Es útil cuando hay desbalance de clases.")
# print("Support: Es el número real de ocurrencias de la clase en el conjunto de prueba.")
# print("Accuracy: Es la proporción de instancias correctamente clasificadas en el conjunto de prueba.")
# print("Macro avg: Es el promedio no ponderado de precision, recall y f1-score para todas las clases.")
# print("Weighted avg: Es el promedio ponderado por el soporte (número de instancias) de precision, recall y f1-score.")

# SVM
print('------------------------------\n')
print('DATOS DEL SUPPORT VECTOR MACHINE (SVM)\n')

svm_classifier = SVC(kernel='sigmoid', C=1.0)
svm_classifier.fit(X_train, np.argmax(y_train, axis=1))

# Datos de test
predictions = svm_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Precisión del SVM: {accuracy * 100:.2f}%')

conf_matrix = confusion_matrix(y_test, predictions)
print(f'Matriz de confusión:\n {conf_matrix}')

mse = mean_squared_error(y_test, predictions)
print(f'Error cuadrático medio: {mse:.2f} \n')

from sklearn.model_selection import cross_val_score

# Entrenar modelo con validación cruzada
scores = cross_val_score(svm_classifier, X_train, np.argmax(y_train, axis=1), cv=5)  # cv es el número de particiones

# Imprimir los scores de cada iteración de validación cruzada
print("Scores de Validación Cruzada:", scores)
print("Promedio de Validación Cruzada:", scores.mean())

Precisión del perceptrón: 87.58%
Matriz de confusión:
 [[893  32]
 [101  45]]
Error cuadratico medio: 0.12 

Precisión del perceptrón: 88.83%
Matriz de confusión:
 [[444  13]
 [ 46  25]]
Error cuadratico medio: 0.11 

------------------------------

DATOS DEL SUPPORT VECTOR MACHINE (SVM)

Precisión del SVM: 84.47%
Matriz de confusión:
 [[421  36]
 [ 46  25]]
Error cuadrático medio: 0.16 

Scores de Validación Cruzada: [0.80930233 0.86448598 0.78971963 0.80373832 0.84579439]
Promedio de Validación Cruzada: 0.822608128667681


### Teoria

Justificar los datos del perceptron respecto a SVM



